| Style Detected         | What to Do                                         |
| ---------------------- | -------------------------------------------------- |
| `\bibitem`             | Already handled via `parse_bibitem_block()`        |
| `\bibliography{}`      | Locate and parse `.bib` files using `bibtexparser` |
| `\bibliographystyle{}` | Optional: record it (not used in parsing directly) |



bibitem 0704.0001

\bibliographystyle{abbrv}. 1912.00003


TODO:

Adding Href handling

Compare with Grobib

In [ ]:
!rm -rf /content/parsed_references.txt

In [ ]:
!rm -rf Latex-papers

In [ ]:
!git clone https://github.com/Luke-Guan-FirstPrinciples/Latex-papers.git

Cloning into 'Latex-papers'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 5), reused 18 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 361.80 KiB | 1.36 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
!pip install bibtexparser pylatexenc

In [ ]:
import os

In [ ]:
!pwd

/content


In [ ]:
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode, LatexMacroNode, LatexCharsNode
import re
import bibtexparser


def clean_latex_text(latex_string):
    """
    Cleans a LaTeX string by removing most LaTeX commands and environments,
    extracting plain text content.

    This function leverages the `pylatexenc` library to robustly parse the LaTeX
    structure. It then traverses the parsed nodes, extracting raw text and
    selectively handling common formatting macros.

    Args:
        latex_string (str): The input LaTeX string.

    Returns:
        str: The cleaned plain text string.
    """
    # Initialize the LatexWalker to parse the input LaTeX string.
    # The LatexWalker breaks down the LaTeX into a hierarchical list of nodes,
    # where each node represents a specific LaTeX element (e.g., characters, macros, environments).
    walker = LatexWalker(latex_string)

    # Get the top-level list of LaTeX nodes from the parsed string.
    # 'pos' indicates the starting position (0 for the beginning of the string),
    # and the third returned value (here '_') is typically a parser-specific state.
    nodelist, pos, _ = walker.get_latex_nodes(pos=0)
    result = []

    def _extract_text(nodes):
        """
        Recursively extracts text content from a list of LaTeX nodes.

        It processes different node types:
        - `LatexCharsNode`: Appends the raw characters to the `result`.
        - `LatexMacroNode`: Handles specific formatting macros like `\\em`,
          `\\textbf`, `\\textit`, and `\\newblock` by recursively extracting
          text from their arguments. It also converts `~` (non-breaking space)
          to a regular space. Unhandled macros are skipped.
        - `LatexEnvironmentNode`: Recursively extracts text from the content
          within the environment.
        - Nodes with a `nodelist` attribute: Recursively processes their sub-nodes.
        """
        for node in nodes:
            if isinstance(node, LatexCharsNode):
                result.append(node.chars)
            elif isinstance(node, LatexMacroNode):
                # Optional: handle common formatting macros
                if node.macroname in {'em', 'textbf', 'textit', 'newblock'}:
                    # For macros with arguments (e.g., \textbf{some text}),
                    # we recursively extract text from their argument list.
                    _extract_text(node.nodeargd.argnlist)
                elif node.macroname == '~':
                    # Replace LaTeX non-breaking space with a regular space.
                    result.append(' ')
                else:
                    # Skip other macros that we don't want in the plain text output.
                    continue
            elif isinstance(node, LatexEnvironmentNode):
                # For environments (e.g., \begin{abstract}...\end{abstract}),
                # we recursively extract text from their content (nodelist).
                _extract_text(node.nodelist)
            elif hasattr(node, 'nodelist'):
                # General case for nodes that might contain a list of sub-nodes.
                _extract_text(node.nodelist)

    # Start the recursive text extraction from the top-level node list.
    _extract_text(nodelist)

    # Final cleanup using regular expressions to remove any remaining LaTeX
    # constructs or artifacts that `pylatexenc` might not have fully stripped
    # or that are part of the broader document structure.
    text = ''.join(result)
    # Remove everything after \end{thebibliography} as it's typically metadata or appendices.
    text = re.sub(r'\\end\{thebibliography\}.*', '', text, flags=re.DOTALL)
    # Remove common commands like \input, \newpage, \end with their arguments.
    text = re.sub(r'\\(input|newpage|end)\{[^}]*\}', '', text)
    # Ensure any remaining ~ are converted to spaces.
    text = re.sub(r'~', ' ', text)
    # Replace multiple spaces with a single space and strip leading/trailing whitespace.
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def parse_bibitem_block(tex_content):
    """
    Parses a block of LaTeX content to extract individual \\bibitem entries.

    It uses regular expressions to find all \\bibitem declarations and
    their corresponding content, then cleans this content using
    `clean_latex_text`.

    Args:
        tex_content (str): The LaTeX content containing \\bibitem entries.

    Returns:
        list[dict]: A list of dictionaries, where each dictionary represents
                    a \\bibitem entry with 'key' (the bibitem key) and
                    'raw' (the cleaned text content of the entry).
    """
    pattern = r'\\bibitem(?:\[[^\]]*\])?\{([^\}]+)\}(.*?)(?=(\\bibitem|\Z))'
    matches = re.findall(pattern, tex_content, re.DOTALL)
    parsed = []

    for key, body, _ in matches:
        clean_body = clean_latex_text(body)
        parsed.append({
            'key': key.strip(),
            'raw': clean_body
        })

    return parsed


def extract_embedded_bibtex_entries(tex_content):
    # Match everything from @TYPE{...} to the closing } of the entry
    bib_entries = re.findall(r'@(?:article|book|inproceedings|misc|phdthesis|techreport|incollection)\s*\{.*?\n\}', tex_content, flags=re.IGNORECASE | re.DOTALL)

    # Convert list into a valid BibTeX string
    bibtex_str = '\n\n'.join(bib_entries)

    # Parse using bibtexparser
    try:
        bib_database = bibtexparser.loads(bibtex_str)
        return bib_database.entries
    except Exception as e:
        print("❌ BibTeX parse error:", e)
        return []


def parse_latex_references(tex_path):
    with open(tex_path, 'r', encoding='utf-8') as f:
        tex_content = f.read()

    # Case: Embedded BibTeX
    if re.search(r'@\w+\s*{', tex_content):
        return extract_embedded_bibtex_entries(tex_content)

    # Case: \bibitem
    if '\\bibitem' in tex_content:
        return parse_bibitem_block(tex_content)

    return []






In [ ]:
from pprint import pformat

# --- Batch Processing Script ---

if __name__ == '__main__':
    folder_path = '/content/Latex-papers'  # Folder containing .tex files
    output_path = '/content/parsed_references.txt'

    with open(output_path, 'w', encoding='utf-8') as out_file:
        for filename in os.listdir(folder_path):
            if filename.endswith('.tex'):
                full_path = os.path.join(folder_path, filename)
                print(f"\n📄 Processing: {filename}")
                try:
                    references = parse_latex_references(full_path)
                    out_file.write(f"File: {filename}\n")
                    out_file.write(pformat(references, indent=2))
                    out_file.write("\n\n" + "="*80 + "\n\n")
                except Exception as e:
                    error_msg = f"❌ Failed to parse {filename}: {e}"
                    print(error_msg)
                    out_file.write(f"{error_msg}\n\n")




📄 Processing: 1905.01664.tex

📄 Processing: 1912.00005.tex

📄 Processing: 1912.00008.tex

📄 Processing: 1912.00010.tex

📄 Processing: 0704.0002.tex

📄 Processing: 0704.0001.tex

📄 Processing: 1905.01024.tex

📄 Processing: 1912.04804.tex

📄 Processing: 0704.0059.tex

📄 Processing: 1912.04803.tex

📄 Processing: 1912.04801.tex

📄 Processing: 1905.10003.tex

📄 Processing: 1912.04802.tex

📄 Processing: 1905.10000.tex

📄 Processing: 1912.00003.tex

📄 Processing: 1912.00001.tex


I asked Gemini:



Show more

Conversation with Gemini
Are the following references the same style:



    'raw': 'authorD.Acosta etal. (collaborationCDF Collaboration), '

           'journalPhys. Rev. Lett. volume95, pages022003 (year2005).'},





    'raw': 'Aldous, D.J. (1985). Exchangeability and related topics. In cole '

           "d't de Probabilits de Saint-Flour XIII—1983, pages 1198. "

           'Springer.'},



    'raw': 'Berg, A.R., Jordn, T.: Algorithms for graph rigidity and scene '

           'analysis. In: Proc. 11th European Symposium on Algorithms (ESA '

           "'03), LNCS, vol. 2832, pp. 7889. (2003)"},





    'raw': "A. D. Aleksandrov and V. V. Strel'tsov, The isoperimetric problem "

           'and estimates of the length of a curve on a surface, Proc. Steklov '

           'Inst. Math. 76 (1965), 67-80; English transi., Proc. Steklov Inst '

           'Math. (1967), 81-99.'},



    'raw': 'Coffman, V., Kundu, J., Wootters, W.K.: Distributed entanglement. '

           'Phys. Rev. A 61, 052306 (2000)'},



    'raw': 'Bakos, G., Noyes, R.W., Kovcs, G., Stanek, K.Z., Sasselov, D.D., '

           'Domsa, I. 2004, , 116, 266'},


The references provided are **not** all in the same style.



* **Reference 1 (Acosta et al.):**
    * **Style indicators:** Author last name, initials, collaboration name, journal abbreviation, volume, pages, year in parentheses. This looks like a common style for physics journals (e.g., APS style).
    * `authorD.Acosta etal. (collaborationCDF Collaboration), journalPhys. Rev. Lett. volume95, pages022003 (year2005).`

* **Reference 2 (Aldous):**
    * **Style indicators:** Author last name, initials, year in parentheses after author, title of work, "In" followed by edited volume title, pages, publisher. This is often seen in conference proceedings or edited books, sometimes resembling a variant of APA or Chicago.
    * `Aldous, D.J. (1985). Exchangeability and related topics. In cole d't de Probabilits de Saint-Flour XIII—1983, pages 1198. Springer.`

* **Reference 3 (Berg, Jordán):**
    * **Style indicators:** Author last name, initials (sometimes separated by a colon), title, "In:" followed by conference name/proceedings, LNCS (Lecture Notes in Computer Science) volume, page range, year in parentheses. This is very common in computer science conference proceedings.
    * `Berg, A.R., Jordn, T.: Algorithms for graph rigidity and scene analysis. In: Proc. 11th European Symposium on Algorithms (ESA '03), LNCS, vol. 2832, pp. 7889. (2003)`

* **Reference 4 (Aleksandrov and Strel'tsov):**
    * **Style indicators:** Initials first, then last name for authors, title of work, journal name (abbreviated), volume, year, page range, and then a separate entry for English translation. This particular style with the "English transi." separated out is less common but indicates a specific format.
    * `A. D. Aleksandrov and V. V. Strel'tsov, The isoperimetric problem and estimates of the length of a curve on a surface, Proc. Steklov Inst. Math. 76 (1965), 67-80; English transi., Proc. Steklov Inst Math. (1967), 81-99.`

* **Reference 5 (Coffman, Kundu, Wootters):**
    * **Style indicators:** Author last name, initials (sometimes with no comma or space between last name and initial), title, journal abbreviation, volume, page number, year in parentheses. Similar to the first one, likely a physics journal style.
    * `Coffman, V., Kundu, J., Wootters, W.K.: Distributed entanglement. Phys. Rev. A 61, 052306 (2000)`

* **Reference 6 (Bakos et al.):**
    * **Style indicators:** Last name, initials, then year, then some numbers that might correspond to volume and page (though the formatting is unusual with double commas and no explicit journal title). This looks incomplete or highly idiosyncratic.
    * `Bakos, G., Noyes, R.W., Kovcs, G., Stanek, K.Z., Sasselov, D.D., Domsa, I. 2004, , 116, 266`

**In summary:** There are variations in author-year placement, punctuation, abbreviations, and the inclusion of elements like "In:" or "Proc." which clearly indicate different citation styles are being used across these examples.

Extract --> Yes

Where do we go from here?

Classify + Recognize --> Not sure

Some failed cases:

1902.00005

\bibliographystyle{IEEEtran}
@ARTICLE{Clarke,
  author = {R. H. Clarke},
  title = {A statistical theory of mobile-radio reception},
  journal = {Bell Syst. Tech. J.},
  year = {1968},
  volume = {47},
  pages = {957-1000},
  publisher = {Bell Syst.}
}

@inproceedings{rezende_stochastic_2014,
	address = {Beijing, China},
	series = {{ICML}'14},
	title = {Stochastic {Backpropagation} and {Approximate} {Inference} in {Deep} {Generative} {Models}},
	url = {http://dl.acm.org/citation.cfm?id=3044805.3045035},
	abstract = {We},
	urldate = {2018-10-05},
	booktitle = {Proceedings of the 31st {International} {Conference} on {International} {Conference} on {Machine} {Learning} - {Volume} 32},
	publisher = {JMLR.org},
	author = {Rezende, Danilo Jimenez and Mohamed, Shakir and Wierstra, Daan},
	year = {2014},
	pages = {II--1278--II--1286}
}

@ARTICLE{Zemen,
  author = {T. Zemen and C. F. Mecklenbrauker and F. Kaltenberger and B. H. Fleury},
  title = {Minimum-Energy Band-Limited Predictor With Dynamic Subspace Selection for Time-Variant Flat-Fading Channels},
  journal = {IEEE Trans. on Signal Process.},
  year = {2007},
  volume = {55},
  pages = {4534-4548}
}


\bibitem

BibTeX-style references

The above has been resolved


1912.04801 -- Citation Section is missing

1912.04804 -- Weird formatting




For the next step, NER, LLM is only way that I could think of


Normal

```


\bibliographystyle{h-physrev-qian}
\begin{thebibliography}{10}


\bibitem{qcd-phase-fluctuations-02}
M.~A. Stephanov,
\newblock Phys. Rev. Lett. {\bf 102} (2009) 032301, arXiv:0809.3450.
```

Failed



```
\providecommand{\href}[2]{#2}\begin{thebibliography}{10}

\bibitem{strominger}
A.~Strominger, ``Superstrings with torsion,'' {\em Nucl. Phys.} {\bf B274}
  (1986)
253.


\bibitem{gauntlett-pakis}
J.~P. Gauntlett and S.~Pakis, ``{The geometry of $D=11$ Killing spinors},''
  {\em JHEP} {\bf 04} (2003) 039,
\href{http://arXiv.org/abs/hep-th/0212008}{{\tt hep-th/0212008}}.

\bibitem{ward}
R.~Ward, ``{Einstein--Weyl spaces and ${\rm SU}(\infty)$ Toda fields},'' {\em
  Classical and Quantum Gravity} {\bf 7} (1990), no.~4, L95.

\end{thebibliography}

\bibliographystyle{at}

```

The \href command is used to create a clickable link to an online resource, in this case, a pre-print on arXiv.org.


Some entries have extra lines between items (like 253. or the \href{} link).



--> is this good enough, requires more testing
\\bibitem(?:\[[^\]]*\])?\{([^\}]+)\}(.*?)(?=(\\bibitem|\Z))




